# Save courses to RDF

## Read data from Firestore

In [1]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

import firebase_admin
from firebase_admin import firestore, initialize_app, credentials

if not firebase_admin._apps:
    cred = credentials.ApplicationDefault()
    initialize_app(cred)

db = firestore.client()

In [2]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [3]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]

Helper Functions

In [4]:
from re import compile
def date_to_xsd_date(string_with_date):
    p = compile('(\d{2}).(\d{2}).(\d{4}).')
    day, month, year = p.search(string_with_date).groups()
    return f'{year}-{month}-{day}'

## Write program to rdf

In [5]:
from rdflib import *
from Aiiso import Aiiso
from University import University
from rdflib.collection import Collection
g = Graph()
g.bind('dc', DC)
g.bind('foaf', FOAF)
g.bind('aiiso', 'http://purl.org/vocab/aiiso/schema#')
g.bind('uni', 'http://www.semanticweb.org/marija/ontologies/2021/university')

In [6]:
program_node = URIRef(program['url'])

g.add((program_node, RDF.type, Aiiso.Programme))
g.add((program_node, Aiiso.Module, Literal(program['naziv'])))
g.add((program_node, University.ESPB, Literal(program['espb'])))
g.add((program_node, University.Semesters, Literal(program['brojSemestara'])))
g.add((program_node, University.Domain, Literal(program['naucnoStrucneOblasti'])))
g.add((program_node, University.LevelOfStudy, Literal(program['nivoStudija'])))
g.add((program_node, University.Field, Literal(program['obrazovnoPolje'])))
g.add((program_node, University.Title, Literal(program['zvanje'])))

g.serialize(format='turtle', destination="program.ttl")
print(g.serialize(format='turtle'))
g.remove((program_node, None, None))

@prefix aiiso: <http://purl.org/vocab/aiiso/schema#> .
@prefix uni: <http://www.semanticweb.org/marija/ontologies/2021/university> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.ftn.uns.ac.rs/875535517/softversko-inzenjerstvo-i-informacione-tehnologije?period=z> a aiiso:Programme ;
    aiiso:Module "softversko inzenjerstvo i informacione tehnologije" ;
    uni:Domain "Elektrotehničko i računarsko inženjerstvo" ;
    uni:ESPB 240 ;
    uni:Field "Tehničko-tehnološke nauke" ;
    uni:LevelOfStudy "Osnovne akademske studije" ;
    uni:Semesters 8 ;
    uni:Title "Diplomirani inženjer softvera (Dipl.inž.softv.)" .




<Graph identifier=Nf0a3755a00ee45df8ddb5cdd5b28aa0d (<class 'rdflib.graph.Graph'>)>

## Write courses to rdf

In [7]:
g1 = Graph()
g1.bind('dc', DC)
g1.bind('foaf', FOAF)
g1.bind('aiiso', 'http://purl.org/vocab/aiiso/schema#')
g1.bind('uni', 'http://www.semanticweb.org/marija/ontologies/2021/university')

In [8]:
def addProfessors(course):
    professors = course['izvodjaciNastave']
    professor_node = BNode()

    for prof in professors:
        list_item = BNode()

        g1.add((course_node, FOAF.Person, list_item))
        g1.add((list_item, FOAF.name, Literal(prof['punoIme'])))
        g1.add((list_item, FOAF.title, Literal(prof['zvanje'])))
        g1.add((list_item, University.TypeOfTeaching, Literal(prof['vidNastave'])))

In [9]:
def addLiterature(course):
    literature = course['literatura']
    literature_node = BNode()

    for lit in literature:
        list_item = BNode()

        g1.add((course_node, University.Literature, list_item))
        g1.add((list_item, DC.creator, Literal(lit['autori'])))
        g1.add((list_item, DC.title, Literal(lit['naziv'])))
        g1.add((list_item, DC.date, Literal(lit['godina'])))
        g1.add((list_item, DC.publisher, Literal(lit['izdavac'])))
        g1.add((list_item, DC.language, Literal(lit['jezik'])))

In [10]:
def addEvaluation(course):
    evaluation = course['formiranjeOcene']
    evaluation_node = BNode()

    for e in evaluation:
        list_item = BNode()

        g1.add((course_node, University.Evaluation, list_item))
        g1.add((list_item, University.Points, Literal(e['brojPoena'])))
        g1.add((list_item, University.Required, Literal(e['obavezna'])))
        g1.add((list_item, University.PreExamination, Literal(e['predispitna'])))
        g1.add((list_item, University.Activity, Literal(e['predmetnaAktivnost'])))

In [11]:
for i, course in enumerate(courses[30:32]):
    dest = "courses" + str(i) + ".ttl"
    course_node = URIRef(course['url'])

    g1.add((course_node, RDF.type, Aiiso.Course))
    g1.add((course_node, DC.title, Literal(course['naziv'])))
    g1.add((course_node, DC.description, Literal(course['sadrzaj'])))
    g1.add((course_node,DC.date, Literal(date_to_xsd_date(course['osnovneInformacije']), datatype=XSD.date)))
    g1.add((course_node, DC.subject, Literal(course['naucnaOblast'])))
    g1.add((course_node, DC.type, Literal(course['kategorija'])))

    g1.add((course_node, University.Purpose, Literal(course['cilj'])))
    g1.add((course_node, University.ESPB, Literal(course['espb'])))
    g1.add((course_node, University.Year, Literal(course['godina'])))
    g1.add((course_node, University.Result, Literal(course['ishod'])))
    g1.add((course_node, University.Methodology, Literal(course['metodologijaIzvodjenjaNastave'])))
    g1.add((course_node, University.Semester, Literal(course['semestar'])))
    
    addProfessors(course)
    addLiterature(course)
    addEvaluation(course)
    
    g1.serialize(format='turtle', destination=dest)
    g1.remove((course_node, None, None))
    
# ttl_dump = g1.serialize(format="turtle", destination='course.ttl')
# print(ttl_dump)